In [236]:
import numpy as np
import pandas as pd
import os
from datetime import date
import sqlalchemy
import pymysql
import re
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', None)

import warnings    # to avoid warning during executions
warnings.filterwarnings("ignore")

In [237]:
file_name = '..\..\..\dataExport\LOGdata\log_2022_4_20.csv'
log_rawDF = pd.read_csv(file_name, sep='*')

In [238]:
feed_temp_ins4 = log_rawDF.query('Time >= 1635246000 and Time <= 1641567595 and Unit =="FPC24" and Type == 2 and Parameter == "Reactor"')

In [239]:
programID_DF = feed_temp_ins4[feed_temp_ins4['Message'].str.contains('Recipe', regex=False, case=False, na=False)]

In [240]:
programID_DF.head()

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
350408,2021-11-2 8:37:25,1635838645,243296,2,FPC24,Reactor,Recipe of Si_1_5de started in FFU_T10 (kg),NaN,0.0857142857142857
350510,2021-11-2 10:57:24,1635847044,749185,2,FPC24,Reactor,Recipe of Complete 3_5de half started in FFU_T10 (kg),NaN,0.514285714285714
350657,2021-11-2 11:33:29,1635849209,742078,2,FPC24,Reactor,Recipe of FeedWaterAcid started in FFU_T10 (kg),NaN,0.257142857142857
352123,2021-11-3 8:33:30,1635924810,738753,2,FPC24,Reactor,Recipe of Si_1_5de started in FFU_T10 (kg),NaN,0.114285714285714
352176,2021-11-3 10:17:39,1635931059,739134,2,FPC24,Reactor,Recipe of Complete 3_5de half started in FFU_T10 (kg),NaN,0.685714285714286


In [241]:
new_DF = log_rawDF.copy()

In [242]:
new_DF_ins4 = new_DF.query('Time >= 1635246000 and Time <= 1641567595')

In [243]:
new_ins4 = new_DF_ins4[new_DF_ins4['Message'].str.contains('FPC24', regex=False, case=False, na=False)]

In [244]:
new_ins4.head()

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
350421,2021-11-2 8:40:21,1635838821,352002,2,FFU,T10,"Dose started for Si_1_5de to Tank = FPC24_Reactor (target dose and current real dose)<table><tr><td width=""12.5%"">DB total dose</td><td width=""12.5%"">0.00000</td><td width=""12.5%"">DB level</td><td width=""12.5%"">0.06067</td><td width=""12.5%"">level</td><td width=""12.5%"">0.04036</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",0.08571,0.00000
350468,2021-11-2 10:49:20,1635846560,869377,2,FFU,T10,"Dose ended at low level for Si_1_5de to Tank = FPC24_Reactor (DB total dose and current real dose)<table><tr><td width=""12.5%"">DB dose</td><td width=""12.5%"">0.09083</td><td width=""12.5%"">DB level</td><td width=""12.5%"">0.00000</td><td width=""12.5%"">level</td><td width=""12.5%"">0.03131</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",0.09083,0.03920
350625,2021-11-2 11:14:52,1635848092,351563,2,FFU,T10,"Dose started for Complete 3_5de half to Tank = FPC24_Reactor (target dose and current real dose)<table><tr><td width=""12.5%"">DB total dose</td><td width=""12.5%"">0.00000</td><td width=""12.5%"">DB level</td><td width=""12.5%"">0.50102</td><td width=""12.5%"">level</td><td width=""12.5%"">0.52379</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",0.51429,0.00000
350629,2021-11-2 11:19:47,1635848387,866793,2,FFU,T10,"Dose ended at low level for Complete 3_5de half to Tank = FPC24_Reactor (DB total dose and current real dose)<table><tr><td width=""12.5%"">DB dose</td><td width=""12.5%"">0.07249</td><td width=""12.5%"">DB level</td><td width=""12.5%"">0.00000</td><td width=""12.5%"">level</td><td width=""12.5%"">0.02936</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",0.50102,0.49444
350664,2021-11-2 11:34:48,1635849288,851779,2,FFU,T10,"Dose started for FeedWaterAcid to Tank = FPC24_Reactor (target dose and current real dose)<table><tr><td width=""12.5%"">DB total dose</td><td width=""12.5%"">0.00000</td><td width=""12.5%"">DB level</td><td width=""12.5%"">0.25719</td><td width=""12.5%"">level</td><td width=""12.5%"">0.28841</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",0.25714,0.00000


In [245]:
no_html_DF =  new_ins4.copy()

In [246]:
#no_html_DF['Message'] = no_html_DF['Message'].str.replace(r'<[^<>]*>', '', regex=True)
no_html_DF['Message'] = no_html_DF['Message'].str.replace(r'<.*?>.*?</.*?>', '', regex=True)

In [247]:
no_html_DF.head()

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
350421,2021-11-2 8:40:21,1635838821,352002,2,FFU,T10,Dose started for Si_1_5de to Tank = FPC24_Reactor (target dose and current real dose),0.08571,0.00000
350468,2021-11-2 10:49:20,1635846560,869377,2,FFU,T10,Dose ended at low level for Si_1_5de to Tank = FPC24_Reactor (DB total dose and current real dose),0.09083,0.03920
350625,2021-11-2 11:14:52,1635848092,351563,2,FFU,T10,Dose started for Complete 3_5de half to Tank = FPC24_Reactor (target dose and current real dose),0.51429,0.00000
350629,2021-11-2 11:19:47,1635848387,866793,2,FFU,T10,Dose ended at low level for Complete 3_5de half to Tank = FPC24_Reactor (DB total dose and current real dose),0.50102,0.49444
350664,2021-11-2 11:34:48,1635849288,851779,2,FFU,T10,Dose started for FeedWaterAcid to Tank = FPC24_Reactor (target dose and current real dose),0.25714,0.00000


In [248]:
no_html_DF.tail()

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
446599,2021-12-29 16:23:29,1640791409,196243,2,FFU,T10,Dose ended at low level for FeedWater to Tank = FPC24_Reactor (DB total dose and current real dose),0.22822,0.21332
446622,2021-12-29 16:30:7,1640791807,667000,2,FFU,T10,Dose started for NaOH_0.1M_rinse to Tank = FPC24_Reactor (target dose and current real dose),0.91429,0.00000
446634,2021-12-29 16:40:9,1640792409,187728,2,FFU,T10,Dose ended at low level for NaOH_0.1M_rinse to Tank = FPC24_Reactor (DB total dose and current real dose),0.91182,0.90214
447338,2021-12-30 9:38:25,1640853505,670297,2,FFU,T10,Dose started for FeedWater to Tank = FPC24_Reactor (target dose and current real dose),1.14286,0.00000
447351,2021-12-30 9:49:54,1640854194,205742,2,FFU,T10,Dose ended at low level for FeedWater to Tank = FPC24_Reactor (DB total dose and current real dose),1.14025,1.14535


In [249]:
feed_tmp1 = log_rawDF.query('Time >= 1635246000 and Time <= 1641567595 and Unit =="FFU" and Type == 2 and Parameter == "T10"')

In [250]:
feed_tmp1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17305 entries, 344717 to 459841
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Date_Time  17305 non-null  object
 1   Time       17305 non-null  int64 
 2   MicroSec   17305 non-null  int64 
 3   Type       17305 non-null  int64 
 4   Unit       17305 non-null  object
 5   Parameter  17305 non-null  object
 6   Message    17305 non-null  object
 7   Value1     13149 non-null  object
 8   Value2     17251 non-null  object
dtypes: int64(3), object(6)
memory usage: 1.3+ MB


In [251]:
feed_tmp2 = feed_tmp1[feed_tmp1.apply(lambda r: r.str.contains('FPC24').any(), axis=1)]

In [252]:
feed_tmp2.head()

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
350409,2021-11-2 8:37:25,1635838645,338961,2,FFU,T10,Remote recipe accepted for,NaN,FPC24_Reactor
350421,2021-11-2 8:40:21,1635838821,352002,2,FFU,T10,"Dose started for Si_1_5de to Tank = FPC24_Reactor (target dose and current real dose)<table><tr><td width=""12.5%"">DB total dose</td><td width=""12.5%"">0.00000</td><td width=""12.5%"">DB level</td><td width=""12.5%"">0.06067</td><td width=""12.5%"">level</td><td width=""12.5%"">0.04036</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",0.08571,0.00000
350468,2021-11-2 10:49:20,1635846560,869377,2,FFU,T10,"Dose ended at low level for Si_1_5de to Tank = FPC24_Reactor (DB total dose and current real dose)<table><tr><td width=""12.5%"">DB dose</td><td width=""12.5%"">0.09083</td><td width=""12.5%"">DB level</td><td width=""12.5%"">0.00000</td><td width=""12.5%"">level</td><td width=""12.5%"">0.03131</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",0.09083,0.03920
350591,2021-11-2 11:13:5,1635847985,840611,2,FFU,T10,Remote recipe accepted for,NaN,FPC24_Reactor
350625,2021-11-2 11:14:52,1635848092,351563,2,FFU,T10,"Dose started for Complete 3_5de half to Tank = FPC24_Reactor (target dose and current real dose)<table><tr><td width=""12.5%"">DB total dose</td><td width=""12.5%"">0.00000</td><td width=""12.5%"">DB level</td><td width=""12.5%"">0.50102</td><td width=""12.5%"">level</td><td width=""12.5%"">0.52379</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",0.51429,0.00000


In [253]:
feed_tmp3 = feed_tmp2[~feed_tmp2.Message.str.contains("ended")]

In [254]:
feed_tmp3.head()

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
350409,2021-11-2 8:37:25,1635838645,338961,2,FFU,T10,Remote recipe accepted for,NaN,FPC24_Reactor
350421,2021-11-2 8:40:21,1635838821,352002,2,FFU,T10,"Dose started for Si_1_5de to Tank = FPC24_Reactor (target dose and current real dose)<table><tr><td width=""12.5%"">DB total dose</td><td width=""12.5%"">0.00000</td><td width=""12.5%"">DB level</td><td width=""12.5%"">0.06067</td><td width=""12.5%"">level</td><td width=""12.5%"">0.04036</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",0.08571,0.00000
350591,2021-11-2 11:13:5,1635847985,840611,2,FFU,T10,Remote recipe accepted for,NaN,FPC24_Reactor
350625,2021-11-2 11:14:52,1635848092,351563,2,FFU,T10,"Dose started for Complete 3_5de half to Tank = FPC24_Reactor (target dose and current real dose)<table><tr><td width=""12.5%"">DB total dose</td><td width=""12.5%"">0.00000</td><td width=""12.5%"">DB level</td><td width=""12.5%"">0.50102</td><td width=""12.5%"">level</td><td width=""12.5%"">0.52379</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",0.51429,0.00000
350658,2021-11-2 11:33:29,1635849209,842095,2,FFU,T10,Remote recipe accepted for,NaN,FPC24_Reactor


In [255]:
feed_tmp3['Micro_1000'] = feed_tmp3['MicroSec']/1000000

In [256]:
feed_tmp3['Time_micro'] = feed_tmp3['Time'] + feed_tmp3['Micro_1000']
feed_tmp3.head()

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2,Micro_1000,Time_micro
350409,2021-11-2 8:37:25,1635838645,338961,2,FFU,T10,Remote recipe accepted for,NaN,FPC24_Reactor,0.338961,1.635839e+09
350421,2021-11-2 8:40:21,1635838821,352002,2,FFU,T10,"Dose started for Si_1_5de to Tank = FPC24_Reactor (target dose and current real dose)<table><tr><td width=""12.5%"">DB total dose</td><td width=""12.5%"">0.00000</td><td width=""12.5%"">DB level</td><td width=""12.5%"">0.06067</td><td width=""12.5%"">level</td><td width=""12.5%"">0.04036</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",0.08571,0.00000,0.352002,1.635839e+09
350591,2021-11-2 11:13:5,1635847985,840611,2,FFU,T10,Remote recipe accepted for,NaN,FPC24_Reactor,0.840611,1.635848e+09
350625,2021-11-2 11:14:52,1635848092,351563,2,FFU,T10,"Dose started for Complete 3_5de half to Tank = FPC24_Reactor (target dose and current real dose)<table><tr><td width=""12.5%"">DB total dose</td><td width=""12.5%"">0.00000</td><td width=""12.5%"">DB level</td><td width=""12.5%"">0.50102</td><td width=""12.5%"">level</td><td width=""12.5%"">0.52379</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",0.51429,0.00000,0.351563,1.635848e+09
350658,2021-11-2 11:33:29,1635849209,842095,2,FFU,T10,Remote recipe accepted for,NaN,FPC24_Reactor,0.842095,1.635849e+09


In [257]:
recipe_tmp1 = log_rawDF.query('Time >= 1635847985 and Time <= 1635848092 and Unit =="FFU" and Type == 2')

In [258]:
recipe_tmp1.head()

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
350590,2021-11-2 11:13:5,1635847985,370133,2,FFU,T10,"Dose ended at low level for Complete 3_5de half to Tank = FPC14_Reactor (DB total dose and current real dose)<table><tr><td width=""12.5%"">DB dose</td><td width=""12.5%"">0.07108</td><td width=""12.5%"">DB level</td><td width=""12.5%"">0.00000</td><td width=""12.5%"">level</td><td width=""12.5%"">0.03775</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",0.67507,0.59843
350591,2021-11-2 11:13:5,1635847985,840611,2,FFU,T10,Remote recipe accepted for,NaN,FPC24_Reactor
350592,2021-11-2 11:13:5,1635847985,877172,2,FFU,feeding,pHCorr product changed (old and new),not found,Complete 3_5de half
350593,2021-11-2 11:13:5,1635847985,880759,2,FFU,feeding,pHCorr product not found in settings DB =&gt; reset to not found,NaN,NaN
350594,2021-11-2 11:13:6,1635847986,387688,2,FFU,feeding,pHCorr product changed (old and new),Complete 3_5de half,not found


In [259]:
recipe_tmp2 = recipe_tmp1[recipe_tmp1['Parameter'].str.startswith('T')]

In [260]:
searchfor = ['recipe', 'ended', 'FPC24']
recipe_tmp3 = recipe_tmp2[recipe_tmp2['Message'].str.contains('|'.join(searchfor))]

In [261]:
recipe_tmp3.head()

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2
350590,2021-11-2 11:13:5,1635847985,370133,2,FFU,T10,"Dose ended at low level for Complete 3_5de half to Tank = FPC14_Reactor (DB total dose and current real dose)<table><tr><td width=""12.5%"">DB dose</td><td width=""12.5%"">0.07108</td><td width=""12.5%"">DB level</td><td width=""12.5%"">0.00000</td><td width=""12.5%"">level</td><td width=""12.5%"">0.03775</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",0.67507,0.59843
350591,2021-11-2 11:13:5,1635847985,840611,2,FFU,T10,Remote recipe accepted for,NaN,FPC24_Reactor
350610,2021-11-2 11:13:40,1635848020,868028,2,FFU,T64,"Dosing ended for NaNO3_2.47M to Tank = FFU_T10 (DB total dose and current real dose)<table><tr><td width=""12.5%"">DB dose</td><td width=""12.5%"">0.00826</td><td width=""12.5%"">DB level</td><td width=""12.5%"">1.02325</td><td width=""12.5%"">level</td><td width=""12.5%"">1.02325</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",0.00826,0.00867
350611,2021-11-2 11:13:45,1635848025,887259,2,FFU,T53,"Dosing ended for KCl_1M to Tank = FFU_T10 (DB total dose and current real dose)<table><tr><td width=""12.5%"">DB dose</td><td width=""12.5%"">0.00843</td><td width=""12.5%"">DB level</td><td width=""12.5%"">0.56452</td><td width=""12.5%"">level</td><td width=""12.5%"">0.56452</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",0.00843,0.00845
350612,2021-11-2 11:13:48,1635848028,902401,2,FFU,T73,"Dosing ended for CaCl2 0.67M to Tank = FFU_T10 (DB total dose and current real dose)<table><tr><td width=""12.5%"">DB dose</td><td width=""12.5%"">0.00539</td><td width=""12.5%"">DB level</td><td width=""12.5%"">1.42583</td><td width=""12.5%"">level</td><td width=""12.5%"">1.42583</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",0.00539,0.00548


In [262]:
log_rawDF['Time_Micro'] = log_rawDF['Time'] + (log_rawDF['MicroSec']/1000000)

In [263]:
log_rawDF.head()

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2,Time_Micro
0,2021-4-20 13:0:13,1618916413,613129,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerdown USB hub",NaN,NaN,1.618916e+09
1,2021-4-20 13:0:57,1618916457,9327,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerdown USB hub",NaN,NaN,1.618916e+09
2,2021-4-20 13:1:38,1618916498,812633,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerdown USB hub",NaN,NaN,1.618916e+09
3,2021-4-20 13:2:20,1618916540,214501,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerdown USB hub",NaN,NaN,1.618917e+09
4,2021-4-20 13:3:3,1618916583,212528,4,FPC11_FPP11,Arduino,"more than 2 communication restarts, try powerdown USB hub",NaN,NaN,1.618917e+09


In [264]:
time_fedd = feed_tmp3['Time_micro'].to_list()

In [265]:
it = iter(time_fedd)
time_tuples = [*zip(it, it)]  # list(zip(it, it))

In [266]:
time_tuples

[(1635838645.338961, 1635838821.352002),
 (1635847985.840611, 1635848092.351563),
 (1635849209.842095, 1635849288.851779),
 (1635924810.839275, 1635925035.351895),
 (1635931059.840101, 1635931190.841621),
 (1635931766.838874, 1635931863.340875),
 (1635950985.342978, 1635951087.841874),
 (1635952125.845963, 1635952279.362048),
 (1635952818.338394, 1635952930.340869),
 (1636105900.347714, 1636106002.352661),
 (1636106793.840415, 1636106945.344659),
 (1636107148.338727, 1636107260.341416),
 (1636109485.839493, 1636109573.345442),
 (1636110407.842122, 1636110538.340703),
 (1636111005.845224, 1636111101.352188),
 (1636362046.343577, 1636362133.340789),
 (1636362985.841908, 1636363116.841464),
 (1636363599.839781, 1636363694.343586),
 (1636506983.338239, 1636507071.351401),
 (1636507889.347259, 1636508019.853006),
 (1636508508.838066, 1636508603.847397),
 (1636550179.8386, 1636550266.344382),
 (1636551130.342527, 1636551260.341013),
 (1636551751.339203, 1636551847.342186),
 (1636593654.83914

In [267]:
appended_feed_data = []
for item in time_tuples:
    data = log_rawDF.query('(Time_Micro >= @item[0] and Time_Micro < @item[1]) and Unit =="FFU" and Type == 2')
    # store DataFrame in list
    appended_feed_data.append(data)
# see pd.concat documentation for more info
appended_feed_data = pd.concat(appended_feed_data)

In [268]:
appended_feed_data.head()

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2,Time_Micro
350409,2021-11-2 8:37:25,1635838645,338961,2,FFU,T10,Remote recipe accepted for,NaN,FPC24_Reactor,1.635839e+09
350410,2021-11-2 8:37:25,1635838645,341266,2,FFU,T10,New product detected,FeedWater,Si_1_5de,1.635839e+09
350411,2021-11-2 8:37:25,1635838645,378997,2,FFU,feeding,pHCorr product changed (old and new),not found,Si_1_5de,1.635839e+09
350413,2021-11-2 8:37:27,1635838647,338671,2,FFU,T10,Recipe of Si_1_5de started (kg),NaN,0.0857142857142857,1.635839e+09
350414,2021-11-2 8:37:27,1635838647,842429,2,FFU,T10,New batch number generated,FFU_T10_20211102_082912,FFU_T10_20211102_083727,1.635839e+09


In [269]:
recipe_tmp2 = appended_feed_data[appended_feed_data['Parameter'].str.startswith('T')]
searchfor = ['recipe', 'ended', 'FPC24', 'Recipe']
recipe_tmp3 = recipe_tmp2[recipe_tmp2['Message'].str.contains('|'.join(searchfor))]

In [270]:
recipe_tmp3.head()

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2,Time_Micro
350409,2021-11-2 8:37:25,1635838645,338961,2,FFU,T10,Remote recipe accepted for,NaN,FPC24_Reactor,1.635839e+09
350413,2021-11-2 8:37:27,1635838647,338671,2,FFU,T10,Recipe of Si_1_5de started (kg),NaN,0.0857142857142857,1.635839e+09
350417,2021-11-2 8:38:1,1635838681,366617,2,FFU,T71,"Dosing ended for CompWater to Tank = FFU_T10 (DB total dose and current real dose)<table><tr><td width=""12.5%"">DB dose</td><td width=""12.5%"">0.06137</td><td width=""12.5%"">DB level</td><td width=""12.5%"">1.43775</td><td width=""12.5%"">level</td><td width=""12.5%"">1.43775</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",0.06137,0.06123,1.635839e+09
350420,2021-11-2 8:40:16,1635838816,363634,2,FFU,T43,"Dosing ended for Na2SiO3_0,212M/EDTA_0,02_fromStock to Tank = FFU_T10 (DB total dose and current real dose)<table><tr><td width=""12.5%"">DB dose</td><td width=""12.5%"">-0.00070</td><td width=""12.5%"">DB level</td><td width=""12.5%"">1.29276</td><td width=""12.5%"">level</td><td width=""12.5%"">1.29276</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",-0.00070,0.00004,1.635839e+09
350591,2021-11-2 11:13:5,1635847985,840611,2,FFU,T10,Remote recipe accepted for,NaN,FPC24_Reactor,1.635848e+09


In [271]:
recipe_tmp2 = appended_feed_data[appended_feed_data['Parameter'].str.startswith('T')]
recipe_tmp3 = recipe_tmp2[recipe_tmp2['Message'].str.contains('ended')]
exclude = ['low level', 'NaOH']
recipe_tmp3 = recipe_tmp3[~recipe_tmp3.Message.str.contains('|'.join(exclude))]

In [272]:
recipe_tmp3.head()

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2,Time_Micro
350417,2021-11-2 8:38:1,1635838681,366617,2,FFU,T71,"Dosing ended for CompWater to Tank = FFU_T10 (DB total dose and current real dose)<table><tr><td width=""12.5%"">DB dose</td><td width=""12.5%"">0.06137</td><td width=""12.5%"">DB level</td><td width=""12.5%"">1.43775</td><td width=""12.5%"">level</td><td width=""12.5%"">1.43775</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",0.06137,0.06123,1.635839e+09
350420,2021-11-2 8:40:16,1635838816,363634,2,FFU,T43,"Dosing ended for Na2SiO3_0,212M/EDTA_0,02_fromStock to Tank = FFU_T10 (DB total dose and current real dose)<table><tr><td width=""12.5%"">DB dose</td><td width=""12.5%"">-0.00070</td><td width=""12.5%"">DB level</td><td width=""12.5%"">1.29276</td><td width=""12.5%"">level</td><td width=""12.5%"">1.29276</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",-0.00070,0.00004,1.635839e+09
350610,2021-11-2 11:13:40,1635848020,868028,2,FFU,T64,"Dosing ended for NaNO3_2.47M to Tank = FFU_T10 (DB total dose and current real dose)<table><tr><td width=""12.5%"">DB dose</td><td width=""12.5%"">0.00826</td><td width=""12.5%"">DB level</td><td width=""12.5%"">1.02325</td><td width=""12.5%"">level</td><td width=""12.5%"">1.02325</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",0.00826,0.00867,1.635848e+09
350611,2021-11-2 11:13:45,1635848025,887259,2,FFU,T53,"Dosing ended for KCl_1M to Tank = FFU_T10 (DB total dose and current real dose)<table><tr><td width=""12.5%"">DB dose</td><td width=""12.5%"">0.00843</td><td width=""12.5%"">DB level</td><td width=""12.5%"">0.56452</td><td width=""12.5%"">level</td><td width=""12.5%"">0.56452</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",0.00843,0.00845,1.635848e+09
350612,2021-11-2 11:13:48,1635848028,902401,2,FFU,T73,"Dosing ended for CaCl2 0.67M to Tank = FFU_T10 (DB total dose and current real dose)<table><tr><td width=""12.5%"">DB dose</td><td width=""12.5%"">0.00539</td><td width=""12.5%"">DB level</td><td width=""12.5%"">1.42583</td><td width=""12.5%"">level</td><td width=""12.5%"">1.42583</td><td width=""12.5%""></td><td width=""12.5%""></td></tr></table>",0.00539,0.00548,1.635848e+09


In [273]:
recipe_tmp3['Message'] = recipe_tmp3['Message'].str.replace(r'<.*?>.*?</.*?>', '', regex=True)

In [274]:
recipe_tmp3['Message'] = recipe_tmp3['Message'].str.replace('Dosing ended for ', '', regex=True)
recipe_tmp3['Message'] = recipe_tmp3['Message'].str.replace('to Tank = FFU_T10', '', regex=True)
recipe_tmp3['Message'] = recipe_tmp3['Message'].str.replace(r'\([^)]*\)', '', regex=True)

In [275]:
recipe_tmp3.head()

,Date_Time,Time,MicroSec,Type,Unit,Parameter,Message,Value1,Value2,Time_Micro
350417,2021-11-2 8:38:1,1635838681,366617,2,FFU,T71,CompWater,0.06137,0.06123,1.635839e+09
350420,2021-11-2 8:40:16,1635838816,363634,2,FFU,T43,"Na2SiO3_0,212M/EDTA_0,02_fromStock",-0.00070,0.00004,1.635839e+09
350610,2021-11-2 11:13:40,1635848020,868028,2,FFU,T64,NaNO3_2.47M,0.00826,0.00867,1.635848e+09
350611,2021-11-2 11:13:45,1635848025,887259,2,FFU,T53,KCl_1M,0.00843,0.00845,1.635848e+09
350612,2021-11-2 11:13:48,1635848028,902401,2,FFU,T73,CaCl2 0.67M,0.00539,0.00548,1.635848e+09


In [276]:
recipe_tmp3['Message'].unique()

array(['CompWater  ', 'Na2SiO3_0,212M/EDTA_0,02_fromStock  ',
       'NaNO3_2.47M  ', 'KCl_1M  ', 'CaCl2 0.67M  ', 'MgSO4_2M  ',
       'NaCl_4,3M  ', 'FeSO4/EDTA 12.05 mM  ', 'H3B03_0,0625M  ',
       'KH2PO4_0.157M  ', 'Vitamins 2000xF  ', 'Micronutrients 400xF  ',
       'Na2SeO3 0.001 mM  '], dtype=object)

In [277]:
recipe_tmp4= recipe_tmp3.drop(['MicroSec', 'Type', 'Value1', 'Unit', 'Parameter', 'Time_Micro'],1)

In [278]:
recipe_tmp4.head()

,Date_Time,Time,Message,Value2
350417,2021-11-2 8:38:1,1635838681,CompWater,0.06123
350420,2021-11-2 8:40:16,1635838816,"Na2SiO3_0,212M/EDTA_0,02_fromStock",0.00004
350610,2021-11-2 11:13:40,1635848020,NaNO3_2.47M,0.00867
350611,2021-11-2 11:13:45,1635848025,KCl_1M,0.00845
350612,2021-11-2 11:13:48,1635848028,CaCl2 0.67M,0.00548


In [279]:
recipe_tmp4['Date'] = pd.to_datetime(recipe_tmp4['Date_Time']).dt.date

In [280]:
recipe_tmp4.head()

,Date_Time,Time,Message,Value2,Date
350417,2021-11-2 8:38:1,1635838681,CompWater,0.06123,2021-11-02
350420,2021-11-2 8:40:16,1635838816,"Na2SiO3_0,212M/EDTA_0,02_fromStock",0.00004,2021-11-02
350610,2021-11-2 11:13:40,1635848020,NaNO3_2.47M,0.00867,2021-11-02
350611,2021-11-2 11:13:45,1635848025,KCl_1M,0.00845,2021-11-02
350612,2021-11-2 11:13:48,1635848028,CaCl2 0.67M,0.00548,2021-11-02


In [290]:
recipe_tmp4 = recipe_tmp4.astype({"Value2": float})

In [291]:
recipe_tmp4.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 366 entries, 350417 to 446617
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date_Time  366 non-null    object 
 1   Time       366 non-null    int64  
 2   Message    366 non-null    object 
 3   Value2     366 non-null    float64
 4   Date       366 non-null    object 
dtypes: float64(1), int64(1), object(3)
memory usage: 17.2+ KB


In [292]:
recipe_tmp6 = recipe_tmp4.pivot_table(index=['Date'], columns='Message', values='Value2', aggfunc=np.sum).reset_index().rename_axis(None, axis=1)


In [293]:
recipe_tmp6.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 14 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Date                                  22 non-null     object 
 1   CaCl2 0.67M                           20 non-null     float64
 2   CompWater                             22 non-null     float64
 3   FeSO4/EDTA 12.05 mM                   20 non-null     float64
 4   H3B03_0,0625M                         20 non-null     float64
 5   KCl_1M                                20 non-null     float64
 6   KH2PO4_0.157M                         20 non-null     float64
 7   MgSO4_2M                              20 non-null     float64
 8   Micronutrients 400xF                  20 non-null     float64
 9   Na2SeO3 0.001 mM                      19 non-null     float64
 10  Na2SiO3_0,212M/EDTA_0,02_fromStock    21 non-null     float64
 11  NaCl_4,3M            

In [294]:
recipe_tmp6.head(60)

,Date,CaCl2 0.67M,CompWater,FeSO4/EDTA 12.05 mM,"H3B03_0,0625M",KCl_1M,KH2PO4_0.157M,MgSO4_2M,Micronutrients 400xF,Na2SeO3 0.001 mM,"Na2SiO3_0,212M/EDTA_0,02_fromStock","NaCl_4,3M",NaNO3_2.47M,Vitamins 2000xF
0,2021-11-02,0.00548,0.67233,0.00853,0.00487,0.00845,0.00869,0.00877,0.00891,0.00006,0.00004,0.07981,0.00867,0.00670
1,2021-11-03,0.01619,2.01813,0.02687,0.01585,0.02590,0.02745,0.02569,0.02603,0.03143,0.04093,0.24028,0.02652,0.02067
2,2021-11-05,0.01644,2.01358,0.02560,0.01568,0.02571,0.02659,0.02607,0.02433,0.03075,0.07368,0.24442,0.02591,0.02094
3,2021-11-08,0.00733,0.89177,0.01740,0.00717,0.01159,0.02313,0.01162,-0.01135,0.01363,0.03253,0.11015,0.01173,0.00914
4,2021-11-10,0.01431,1.79133,0.02435,0.01439,0.02292,0.02084,0.02282,0.02347,0.02733,0.06492,0.21736,0.02281,0.01865
5,2021-11-11,0.01380,1.79239,0.02274,0.01410,0.02220,0.02199,0.02250,0.02254,0.02716,0.03286,0.21567,0.02294,0.01791
6,2021-11-16,0.00880,1.11985,0.01438,0.00856,0.01423,0.01432,0.01438,0.01432,0.01743,0.04032,0.13496,0.01438,0.01141
7,2021-11-17,NaN,1.20403,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.08078,NaN,NaN,NaN
8,2021-12-02,0.00723,0.89748,0.01175,0.00733,0.01122,0.01145,0.01169,0.01141,0.01365,0.00212,0.10833,0.01207,0.00935
9,2021-12-05,0.00704,0.89241,0.01264,0.00708,0.01139,0.01154,0.01150,0.01147,0.01379,0.03250,0.10901,0.01158,0.00910


In [295]:
recipe_table = recipe_tmp6.ffill().bfill()

In [296]:
recipe_table.head(60)

,Date,CaCl2 0.67M,CompWater,FeSO4/EDTA 12.05 mM,"H3B03_0,0625M",KCl_1M,KH2PO4_0.157M,MgSO4_2M,Micronutrients 400xF,Na2SeO3 0.001 mM,"Na2SiO3_0,212M/EDTA_0,02_fromStock","NaCl_4,3M",NaNO3_2.47M,Vitamins 2000xF
0,2021-11-02,0.00548,0.67233,0.00853,0.00487,0.00845,0.00869,0.00877,0.00891,0.00006,0.00004,0.07981,0.00867,0.00670
1,2021-11-03,0.01619,2.01813,0.02687,0.01585,0.02590,0.02745,0.02569,0.02603,0.03143,0.04093,0.24028,0.02652,0.02067
2,2021-11-05,0.01644,2.01358,0.02560,0.01568,0.02571,0.02659,0.02607,0.02433,0.03075,0.07368,0.24442,0.02591,0.02094
3,2021-11-08,0.00733,0.89177,0.01740,0.00717,0.01159,0.02313,0.01162,-0.01135,0.01363,0.03253,0.11015,0.01173,0.00914
4,2021-11-10,0.01431,1.79133,0.02435,0.01439,0.02292,0.02084,0.02282,0.02347,0.02733,0.06492,0.21736,0.02281,0.01865
5,2021-11-11,0.01380,1.79239,0.02274,0.01410,0.02220,0.02199,0.02250,0.02254,0.02716,0.03286,0.21567,0.02294,0.01791
6,2021-11-16,0.00880,1.11985,0.01438,0.00856,0.01423,0.01432,0.01438,0.01432,0.01743,0.04032,0.13496,0.01438,0.01141
7,2021-11-17,0.00880,1.20403,0.01438,0.00856,0.01423,0.01432,0.01438,0.01432,0.01743,0.08078,0.13496,0.01438,0.01141
8,2021-12-02,0.00723,0.89748,0.01175,0.00733,0.01122,0.01145,0.01169,0.01141,0.01365,0.00212,0.10833,0.01207,0.00935
9,2021-12-05,0.00704,0.89241,0.01264,0.00708,0.01139,0.01154,0.01150,0.01147,0.01379,0.03250,0.10901,0.01158,0.00910


In [ ]:
'''
object_columns = feed_tmp1.select_dtypes(include=['object']).columns
object_columns
'''

'''
recipe_tmp5 = recipe_tmp4[['Date_Time','Date', 'Message', 'Value2']]
recipe_tmp5.head()
'''
''' 
date_dt = ['Date']
new_df = recipe_tmp4.set_index(['Message']).unstack('Message')
new_df.data.rename_axis([None], axis=1).reset_index()
'''

''' 
new_df.data.rename_axis([None], axis=1).reset_index()
print (df.pivot_table(index=['meta1','meta2'], 
                      columns='name', 
                      values='data', 
                      aggfunc='first')
         .reset_index()
         .rename_axis(None, axis=1))

'''

Index(['Date_Time', 'Unit', 'Parameter', 'Message', 'Value1', 'Value2'], dtype='object')